### Esta sección aborda la tarea más compleja de la estimación de pose, desde la definición conceptual de un esqueleto anatómico para salmones hasta la implementación del código y la extracción de métricas cinemáticas.

In [3]:

import json
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO

In [ ]:

# Ejemplo de uso:
# La estructura de carpetas y el archivo data.yaml son análogos al caso de segmentación.
# convert_coco_kpts_to_yolo_pose('/path/to/dataset/annotations/person_keypoints_train.json', '/path/to/yolo_pose_dataset/train/')


def convert_coco_kpts_to_yolo_pose(coco_json_path, save_dir, use_segmentation_for_bbox=False):
    """
    Convierte anotaciones de pose en formato COCO JSON al formato YOLOv8-Pose.
    """
    save_path = Path(save_dir)
    labels_path = save_path / 'labels'
    labels_path.mkdir(parents=True, exist_ok=True)

    with open(coco_json_path, 'r') as f:
        data = json.load(data)

    images_info = {img['id']: {'file_name': img['file_name'], 'height': img['height'], 'width': img['width']} for img in data['images']}
    categories = {cat['id']: i for i, cat in enumerate(data['categories'])}
    num_kpts = len(data['categories']['keypoints'])

    annotations_by_image = {}
    for ann in data['annotations']:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] =
        annotations_by_image[img_id].append(ann)

    print("Iniciando conversión de anotaciones de pose...")
    for img_id, anns in tqdm(annotations_by_image.items()):
        img_info = images_info[img_id]
        img_h, img_w = img_info['height'], img_info['width']
        
        if img_h == 0 or img_w == 0:
            continue

        label_name = Path(img_info['file_name']).stem + '.txt'
        label_path = labels_path / label_name

        with open(label_path, 'w') as f:
            for ann in anns:
                if 'keypoints' not in ann or ann['num_keypoints'] == 0:
                    continue

                class_index = categories[ann['category_id']]
                
                # Bounding box
                x, y, w, h = ann['bbox']
                xc = (x + w / 2) / img_w
                yc = (y + h / 2) / img_h
                wn = w / img_w
                hn = h / img_h

                line_parts = [str(class_index), str(xc), str(yc), str(wn), str(hn)]

                # Keypoints
                kpts = ann['keypoints']
                for i in range(num_kpts):
                    kpt_x = kpts[i * 3]
                    kpt_y = kpts[i * 3 + 1]
                    vis = kpts[i * 3 + 2]

                    # YOLO format no usa la bandera de visibilidad directamente,
                    # pero los puntos no visibles (v=0) se ponen en (0,0)
                    if vis == 0:
                        nx, ny = 0, 0
                    else:
                        nx = kpt_x / img_w
                        ny = kpt_y / img_h
                    
                    line_parts.extend([str(nx), str(ny)])
                
                f.write(" ".join(line_parts) + '\n')
    
    print(f"Conversión completada. Las etiquetas se guardaron en: {labels_path}")

In [ ]:
# Cargar un modelo de estimación de pose pre-entrenado YOLOv8n-pose.pt
model = YOLO('yolov8n-pose.pt')

# Entrenar el modelo
print("Iniciando el entrenamiento del modelo de estimación de pose...")
results = model.train(
    data='path/to/salmon_pose_dataset/data.yaml',
    epochs=150,
    imgsz=640,
    batch=16,
    patience=30,
    name='yolov8n_salmon_pose'
)
print("Entrenamiento completado.")
print(f"El mejor modelo se guardó en: {results.save_dir}")